In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px
from collections import Counter
import time

from bertopic import BERTopic
from itertools import chain
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from umap import UMAP
from hdbscan import HDBSCAN

In [2]:
# email_passage_splits = [len(embedding) for embedding in email_embeddings_np]

# passage_mapping ={}
# j = 0
# for i in range(len(email_passage_splits)):
#     passages_count = email_passage_splits[i]
#     passage_mapping[i] = cluster_labels[j:j+passages_count]
#     j += passages_count

In [3]:
podcast_df = pd.read_pickle('Podcast/podcast_data_processed.pkl')

In [4]:
flattened_passages = list(chain.from_iterable(podcast_df['Passages']))

In [18]:
len(flattened_passages)

656040

In [5]:
# import random
# samples = random.sample(flattened_passages, 50000)

In [12]:
umap_model = UMAP(n_neighbors=4, min_dist=0.25)
hdbscan_model = HDBSCAN(min_cluster_size=125, gen_min_span_tree=True, prediction_data=True)

In [13]:
topic_model = BERTopic(n_gram_range=(1, 3), top_n_words=10,
                      umap_model=umap_model, hdbscan_model=hdbscan_model, verbose=True)
topics, probs = topic_model.fit_transform(flattened_passages)

Batches:   0%|          | 0/20502 [00:00<?, ?it/s]

2023-10-18 21:19:22,041 - BERTopic - Transformed documents to Embeddings
2023-10-18 22:12:55,578 - BERTopic - Reduced dimensionality
python(28346) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(28347) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


python(28348) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(28349) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


2023-10-18 22:14:19,371 - BERTopic - Clustered reduced embeddings


In [14]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,190214,-1_the_to_and_you,"[the, to, and, you, of, that, in, is, it, this]",[But the Republican leader Kevin McCarthy says...
1,0,279430,0_the_and_to_that,"[the, and, to, that, you, of, in, is, it, this]",[I can tell you exactly what you're going to s...
2,1,26837,1_ukraine_putin_russia_the,"[ukraine, putin, russia, the, to, and, of, in,...","[Instead, it turned out that Putin went for br..."
3,2,19961,2_covid_vaccine_the_that,"[covid, vaccine, the, that, and, to, of, you, ...","[I was going to say, when you when you look at..."
4,3,16807,3_energy_oil_gas_the,"[energy, oil, gas, the, to, and, of, we, is, p...",[And since those are the only two factors caus...
...,...,...,...,...,...
144,143,145,143_quinton_fedex_quinton here makes_quinton very,"[quinton, fedex, quinton here makes, quinton v...",[Quinton here makes sustainable clothing. Yep....
145,144,137,144_title_home_your_home title,"[title, home, your, home title, your homes tit...",[Like you got to pay for the legal fees. You c...
146,145,137,145_military_the military_you_to,"[military, the military, you, to, the, in, and...",[You can barely find me now on YouTube. You ca...
147,146,130,146_marriage_divorce_babylon_married,"[marriage, divorce, babylon, married, babylon ...",[There is no question that younger Americans a...


In [15]:
topic_model.visualize_hierarchy()

In [16]:
topic_model.visualize_hierarchy().write_html('Podcast/podcast-bert-clust.html')

In [17]:
topic_model.get_topic_info().to_pickle('Podcast/podcast-bert-clust.pkl')

In [11]:
# # Define a list of parameters to try for UMAP
# umap_params = [
#     {'n_neighbors': 3,'min_dist': 0.05},
#     {'n_neighbors': 3,'min_dist': 0.1},
#     {'n_neighbors': 6,'min_dist': 0.05}
# ]

# # Define a list of parameters to try for HDBSCAN
# hdbscan_params = [
#     {'min_cluster_size': 40},
#     {'min_cluster_size': 50}
# ]

# # Loop over the parameter combinations and fit BERTopic models
# for umap_param in umap_params:
#     for hdbscan_param in hdbscan_params:
#         # Create UMAP and HDBSCAN models with the current parameter combination
#         umap_model = UMAP(**umap_param)
#         hdbscan_model = HDBSCAN(**hdbscan_param, gen_min_span_tree=True, prediction_data=True)

#         # Fit a BERTopic model with the current parameter combination
#         model = BERTopic(
#             umap_model=umap_model,
#             hdbscan_model=hdbscan_model,
#             top_n_words=10,
#             language='english',
#             calculate_probabilities=True,
#             verbose=True,
#             n_gram_range=(1, 2)
#         )
#         topics, probs = model.fit_transform(flattened_passages)
#         model.get_topic_info().to_pickle(f'Email/email_model_umap_{umap_param}_hdbscan_{hdbscan_param}.pkl')

#         # Visualize the hierarchy and save the figure to an HTML file
#         fig = model.visualize_hierarchy()
#         fig.write_html(f'Email/email_model_umap_{umap_param}_hdbscan_{hdbscan_param}.html')